# Training Binarized NN with MaxSAT

In [ ]:
# Imports
import numpy as np
import itertools
import operator
from pysat.formula import WCNF
from pysat.examples.fm import FM
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Binarized NN without hidden layer

### Data generation with different binary fucntions

In [2]:
N = 5 # Input neurons
M = 5 # Output neurons
p_test = 0.3 # Percentage of test data

Use all possible combinations for dataset in case of the input size N

In [3]:
np.random.seed(42)

X = list(itertools.product([-1, 1], repeat=N))
np.random.shuffle(X)

Logical fucntions of different complexity for the input size N=5 and the output size M=5. Consider to adjust them for the different dimentionality.

In [4]:
def logical_function_1(X):
    return [
        (
            x[3],  
            x[1] or x[2],  # OR operation
            x[0], 
            -x[3],  # NOT operation
            x[3] and x[4]  # AND operation
        )
        for x in X
    ]


def logical_function_2(X):
    return [
        (
            x[0] and x[1],  # AND operation
            x[2] or x[3],  # OR operation
            (x[2] and -x[3]) or (-x[2] and x[3]),  # XOR 
            -x[3],  # NOT operation
            (x[4] and x[0]) or x[1]  # Complex OR-AND combination
        )
        for x in X
    ]


def logical_function_3(X):
    return [
        (
            (x[0] and x[1]) or ((x[2] and -x[3]) or (-x[2] and x[3])),  # Combination of AND, OR, XOR
            (x[1] or x[2]) and (-x[3] or x[4]),  # OR and NOT combination
            x[0] or (x[1] and x[2] and -x[3]) or (-x[1] and -x[2] and x[3]),  # XOR mixed with AND and OR
            (x[3] and x[4]) or (-x[0] and -x[1]) or (x[0] and x[1]),  # Mix of XOR, NOT, and OR
            ((x[2] and x[3]) or (x[4] and -x[0]) or (-x[4] and x[0])) and x[1]  # More nested operations
        )
        for x in X
    ]

In [5]:
# Generate outputs for different complexity levels
Y1 = logical_function_1(X)
Y2 = logical_function_2(X)
Y3 = logical_function_3(X)

In [6]:
# Split data into train and test sets
X_train, X_test, Y_train1, Y_test1 = train_test_split(X, Y1, test_size=p_test, random_state=42)
X_train, X_test, Y_train2, Y_test2 = train_test_split(X, Y2, test_size=p_test, random_state=42)
X_train, X_test, Y_train3, Y_test3 = train_test_split(X, Y3, test_size=p_test, random_state=42)

In [7]:
# Train size
train_size = len(X_train)

### SAT Encoding by blocks

In [8]:
# Function to compute SAT encoding for various indices
def SAT_encoding(i, n, N, offset = 0):
   """
   Computes the SAT encoding index for inputs, weights, or other components.

   Parameters:
   i: int, Represents the current index (e.g., sample index in a training set).
   n int, Represents the specific neuron or weight index within the layer.
   N : int, The total number of elements in a set (e.g., number of input neurons).
   offset : int, An offset to shift index values appropriately for different sections (default is 0).

   Returns:
   int, The computed SAT encoding index.
   """
   return offset + i*N + n + 1

# Function to encode input neurons
def x_encod_i_n(i, n):
    """
    Encodes the SAT index for an input neuron.

    Parameters:
    i : int, Index of the training sample.
    n : int, Index of the input neuron.

    Returns:
    int, The computed SAT index for input neurons.
    """
    return SAT_encoding(i, n, N, offset = 0)

# Function to encode weights connecting neurons
def w_encode_n_p(n, m):
    """
    Encodes the SAT index for weights between neurons.

    Parameters:
    n : int, Index of the neuron in the previous layer.
    m : int, Index of the neuron in the next layer.

    Returns:
    int, The computed SAT index for the weight connecting neurons.
    """
    return SAT_encoding(n, m, M, offset = train_size * N)


### Weighted CNF for training

We add both hard (input neurons) and soft (the punishement for misclassification) constarints

In [9]:
wcnf = WCNF()

In [10]:
# Hard constarints
for i in range(train_size):
    for j in range(N):
        if X_train[i][j] > 0:
            wcnf.append([x_encod_i_n(i,j)])
        else:
            wcnf.append([-x_encod_i_n(i,j)])

print(wcnf.hard)

[[-1], [2], [-3], [-4], [-5], [-6], [-7], [-8], [-9], [10], [-11], [12], [-13], [-14], [15], [-16], [17], [18], [-19], [20], [21], [-22], [-23], [-24], [-25], [26], [-27], [-28], [29], [-30], [-31], [32], [33], [34], [35], [36], [37], [-38], [-39], [-40], [-41], [-42], [43], [44], [45], [46], [-47], [-48], [-49], [50], [51], [52], [-53], [54], [55], [-56], [57], [-58], [59], [-60], [61], [-62], [63], [64], [-65], [66], [67], [-68], [69], [-70], [-71], [-72], [73], [74], [-75], [76], [-77], [78], [-79], [80], [81], [82], [-83], [-84], [85], [-86], [-87], [88], [-89], [-90], [-91], [92], [-93], [94], [95], [-96], [97], [98], [99], [-100], [-101], [-102], [-103], [104], [105], [106], [107], [108], [109], [-110]]


In [11]:
# More than half inputs for soft constarints
N_half = N//2+1

In [12]:
input_comb = list(itertools.combinations(range(N), N_half)) # More than half of inputs
neg_comb = list(itertools.product([-1, 1],  repeat = N_half)) # With negation or not

soft_combs = list(itertools.product(input_comb, neg_comb)) # Combinations for soft constraints

In [ ]:
# Soft constraints
def add_soft_const(wcnf, y_train):
    """
    Adds soft constraints to the weighted CNF formulation.
    
    Parameters:
    - wcnf: WCNF object with hard constraints already added.
    - y_train: Training labels for the dataset.

    The function enforces that the weight assignments align with the expected output,
    encouraging correct classification while allowing some flexibility (soft constraints).
    """
    for i in range(train_size):   # Iterate over each training sample
        for m in range(M):   # Iterate over each output neuron

            # If the expected output is 1, we enforce positive influence
            if y_train[i][m] == 1:
                for inputs, signs in soft_combs:   # Iterate over all possible input combinations
                    clause = []
                    for n_num, n in enumerate(inputs):   # Iterate over selected input neurons
                        clause.append(signs[n_num]*x_encod_i_n(i,n))   # Encode input neurons
                        clause.append(-signs[n_num]*w_encode_n_p(n,m))   # Ensure weights align with input
                    wcnf.append(clause, weight=1)

            # If the expected output is -1, we enforce negative influence
            if y_train[i][m] == -1:
                for inputs, signs in soft_combs:   # Iterate over all possible input combinations
                    clause = []
                    for n_num, n in enumerate(inputs):  # Iterate over selected input neurons
                        clause.append(signs[n_num]*x_encod_i_n(i,n))   # Encode input neurons
                        clause.append(signs[n_num]*w_encode_n_p(n,m))    # Ensure weights align with input
                    wcnf.append(clause, weight=1)                

In [14]:
wcnf_cp = wcnf.copy()
add_soft_const(wcnf_cp, Y_train1)
print("Number of soft constraints: ", len(wcnf_cp.soft))

Number of soft constraints:  8800


In [15]:
solver = FM(wcnf_cp)
solver.compute()
model = solver.model
print("Solver cost: ", solver.cost) 

Solver cost:  27


In [16]:
# Weights
W = np.sign(model[train_size*N :]).reshape(N, M)

In [17]:
W

array([[-1, -1,  1,  1, -1],
       [ 1,  1, -1, -1, -1],
       [ 1, -1, -1, -1, -1],
       [ 1,  1,  1, -1, -1],
       [-1, -1, -1,  1,  1]])

In [18]:
# Predictions
pred_train = np.sign(np.matmul(X_train, W))
pred_test = np.sign(np.matmul(X_test, W))

In [19]:
print(f"Training Accuracy: {accuracy_score(list(itertools.chain(*Y_train1)), list(itertools.chain(*pred_train)))}")
print(f"Test Accuracy: {accuracy_score(list(itertools.chain(*Y_test1)), list(itertools.chain(*pred_test)))}")

Training Accuracy: 0.7545454545454545
Test Accuracy: 0.54


For all functions:

In [20]:
Y_test_list = [Y_test1, Y_test2, Y_test3]
for i, y_train in enumerate([Y_train1, Y_train2, Y_train3]):
    print(f"Function {i + 1}:")

    wcnf_cp = wcnf.copy()
    add_soft_const(wcnf_cp, y_train)

    solver = FM(wcnf_cp)
    solver.compute()
    model = solver.model
    print("Solver cost: ", solver.cost) 

    W = np.sign(model[train_size*N :]).reshape(N, M)

    pred_train = np.sign(np.matmul(X_train, W))
    pred_test = np.sign(np.matmul(X_test, W))

    print(f"Training Accuracy: {accuracy_score(list(itertools.chain(*y_train)), list(itertools.chain(*pred_train)))}")
    print(f"Test Accuracy: {accuracy_score(list(itertools.chain(*Y_test_list[i])), list(itertools.chain(*pred_test)))}\n")


Function 1:
Solver cost:  27
Training Accuracy: 0.7545454545454545
Test Accuracy: 0.54

Function 2:
Solver cost:  27
Training Accuracy: 0.7545454545454545
Test Accuracy: 0.54

Function 3:
Solver cost:  26
Training Accuracy: 0.7636363636363637
Test Accuracy: 0.52



## Binarized NN with hidden layer

### Data generation with different binary fucntions

In [21]:
N = 5 # Input neurons
H = 3 # Hidden neurons
M = 3 # Output neurons -> have to reduce the number, otherwise not enough RAM to train (15 Gb available)
p_test = 0.3 # Percentage of test data

Use all possible combinations for dataset in case of the input size N

In [22]:
np.random.seed(42)

X = list(itertools.product([-1, 1], repeat=N))
np.random.shuffle(X)

Logical fucntions of different complexity for the input size N=5 and the output size M=5. Consider to adjust them for the different dimentionality.

In [23]:
def logical_function_1(X):
    return [
        (
            x[1] or x[2],  # OR operation
            -x[0],  # NOT operation
            x[3] and x[4]  # AND operation
        )
        for x in X
    ]


def logical_function_2(X):
    return [
        (
            x[4] and x[1],  # AND operation
            (x[2] and -x[3]) or (-x[2] and x[3]),  # XOR 
            (x[4] and x[0]) or x[1]  # Complex OR-AND-NOT combination
        )
        for x in X
    ]


def logical_function_3(X):
    return [
        (
            (x[0] and x[1]) or ((x[2] and -x[3]) or (-x[2] and x[3])),  # Combination of AND, OR, XOR
            x[0] or (x[1] and x[2] and -x[3]) or (-x[1] and -x[2] and x[3]),  # XOR mixed with AND and OR
            ((x[2] and x[3]) or (x[4] and -x[0]) or (-x[4] and x[0])) and x[1]  # More nested operations
        )
        for x in X
    ]

In [24]:
# Generate outputs for different complexity levels
Y1 = logical_function_1(X)
Y2 = logical_function_2(X)
Y3 = logical_function_3(X)

In [25]:
# Split data into train and test sets
X_train, X_test, Y_train1, Y_test1 = train_test_split(X, Y1, test_size=p_test, random_state=42)
X_train, X_test, Y_train2, Y_test2 = train_test_split(X, Y2, test_size=p_test, random_state=42)
X_train, X_test, Y_train3, Y_test3 = train_test_split(X, Y3, test_size=p_test, random_state=42)

In [26]:
# Train size
train_size = len(X_train)

### SAT Encoding by blocks

In [27]:
# Functions to encode weights connecting neurons

# Input -> Hidden
def w1_encode_n_h(n, h):
    """
    Encodes the SAT index for weights between neurons.

    Parameters:
    n : int, Index of the neuron in the previous layer.
    h : int, Index of the neuron in the next layer.

    Returns:
    int, The computed SAT index for the weight connecting neurons.
    """
    return SAT_encoding(n, h, H, offset = train_size * N)

# Hidden -> Output
def w2_encode_h_m(h, m):
    """
    Encodes the SAT index for weights between neurons.

    Parameters:
    h : int, Index of the neuron in the previous layer.
    m : int, Index of the neuron in the next layer.

    Returns:
    int, The computed SAT index for the weight connecting neurons.
    """
    return SAT_encoding(h, m, M, offset = train_size * N + N * H)


### Weighted CNF for training

We add both hard (input neurons) and soft (the punishement for misclassification) constarints

In [28]:
wcnf = WCNF()

In [29]:
# Hard constarints
for i in range(train_size):
    for j in range(N):
        if X_train[i][j] > 0:
            wcnf.append([x_encod_i_n(i,j)])
        else:
            wcnf.append([-x_encod_i_n(i,j)])
            
print(wcnf.hard)

[[-1], [2], [-3], [-4], [-5], [-6], [-7], [-8], [-9], [10], [-11], [12], [-13], [-14], [15], [-16], [17], [18], [-19], [20], [21], [-22], [-23], [-24], [-25], [26], [-27], [-28], [29], [-30], [-31], [32], [33], [34], [35], [36], [37], [-38], [-39], [-40], [-41], [-42], [43], [44], [45], [46], [-47], [-48], [-49], [50], [51], [52], [-53], [54], [55], [-56], [57], [-58], [59], [-60], [61], [-62], [63], [64], [-65], [66], [67], [-68], [69], [-70], [-71], [-72], [73], [74], [-75], [76], [-77], [78], [-79], [80], [81], [82], [-83], [-84], [85], [-86], [-87], [88], [-89], [-90], [-91], [92], [-93], [94], [95], [-96], [97], [98], [99], [-100], [-101], [-102], [-103], [104], [105], [106], [107], [108], [109], [-110]]


In [30]:
# More than half inputs for soft constarints
N_half = N//2+1
H_half = H//2+1

In [31]:
input_comb = list(itertools.combinations(range(N), N_half)) # More than half of inputs
neg_comb_inp = list(itertools.product([-1, 1],  repeat = N_half)) # With negation or not
soft_combs_inp = list(itertools.product(input_comb, neg_comb_inp)) # Combinations for soft constraints

hidden_comb = list(itertools.combinations(range(H), H_half)) # More than half of hidden neurons
neg_comb_hid = list(itertools.product([-1, 1],  repeat = H_half)) # With negation or not
soft_combs_hid = list(itertools.product(hidden_comb, neg_comb_hid)) # Combinations for soft constraints

In [ ]:
# Soft constraints
def add_soft_const(wcnf, y_train):
    """
    Adds soft constraints to the Weighted CNF formulation for a Binarized Neural Network with a hidden layer.

    Parameters:
    - wcnf: WCNF object with hard constraints already added.
    - y_train: Training labels for the dataset.

    The function enforces that the weight assignments allow for correct activations 
    at both the hidden and output layers while minimizing constraint violations.
    """
    for i in range(train_size):   # Iterate over each training sample
        for m in range(M):   # Iterate over each output neuron

             # Checks correctness of outputs
            if y_train[i][m] not in [1, -1]:
                continue  
            
            sign_m = y_train[i][m]    # Store the sign for easier reference

            for inputs_h, signs_h in soft_combs_hid:   # Iterate over hidden neuron activation patterns
                w2_clause_parts = []    # Store parts of the final clause
                
                for h_num, h in enumerate(inputs_h):   # Iterate over selected hidden neurons
                    h_clauses = []   # Store clauses for hidden neuron activation
                    
                    for inputs_n, signs_n in soft_combs_inp:    # Iterate over input combinations affecting hidden neurons
                        h_clause = []

                        for n_num, n in enumerate(inputs_n):    # Iterate over selected input neurons
                            h_clause.append(signs_n[n_num]*x_encod_i_n(i,n))    # Encode input neuron

                            # Handle sign logic for weight between input and hidden layer
                            if signs_h[h_num] == 1:
                                h_clause.append(-signs_n[n_num] * w1_encode_n_h(n, h))   # Negative for correct alignment
                            else:
                                h_clause.append(signs_n[n_num] * w1_encode_n_h(n, h))   # Positive for incorrect alignment

                        h_clauses.append(h_clause)

                    # Append W2 encoding (hidden to output) with the correct sign
                    h_clauses_plus_w2 = [sublist + [-sign_m * signs_h[h_num] * w2_encode_h_m(h, m)] for sublist in h_clauses]
                    w2_clause_parts.append(h_clauses_plus_w2)
                
                # Generate the final set of clauses by combining hidden layer constraints
                clauses = [sum(combo, []) for combo in itertools.product(*w2_clause_parts)]
                
                # Append each generated clause to the weighted CNF
                for clause in clauses:
                    wcnf.append(clause, weight=1)


In [33]:
wcnf_cp = wcnf.copy()
add_soft_const(wcnf_cp, Y_train1)
print("Number of soft constraints: ", len(wcnf_cp.soft))

Number of soft constraints:  5068800


In [34]:
wcnf_cp.soft[0]

[-1, -111, -2, -114, -3, -117, 126, -1, -112, -2, -115, -3, -118, 129]

In [35]:
solver = FM(wcnf_cp)
solver.compute()
model = solver.model
print(solver.cost) 

2


In [36]:
# Weights
W1 = np.sign(model[train_size*N : train_size*N + N*H]).reshape(N, H)
W2 = np.sign(model[train_size*N + N*H :]).reshape(H, M)

In [37]:
pred_train = np.sign(np.matmul(np.matmul(X_train, W1), W2))
pred_test = np.sign(np.matmul(np.matmul(X_test, W1), W2))

In [38]:
print(f"Training Accuracy: {accuracy_score(list(itertools.chain(*Y_train1)), list(itertools.chain(*pred_train)))}")
print(f"Test Accuracy: {accuracy_score(list(itertools.chain(*Y_test1)), list(itertools.chain(*pred_test)))}")

Training Accuracy: 1.0
Test Accuracy: 0.8


For all functions:

In [39]:
Y_test_list = [Y_test1, Y_test2, Y_test3]
for i, y_train in enumerate([Y_train1, Y_train2, Y_train3]):
    print(f"Function {i + 1}:")

    wcnf_cp = wcnf.copy()
    add_soft_const(wcnf_cp, y_train)

    solver = FM(wcnf_cp)
    solver.compute()
    model = solver.model
    print("Solver cost: ", solver.cost) 

    W1 = np.sign(model[train_size*N : train_size*N + N*H]).reshape(N, H)
    W2 = np.sign(model[train_size*N + N*H :]).reshape(H, M)


    pred_train = np.sign(np.matmul(np.matmul(X_train, W1), W2))
    pred_test = np.sign(np.matmul(np.matmul(X_test, W1), W2))

    print(f"Training Accuracy: {accuracy_score(list(itertools.chain(*y_train)), list(itertools.chain(*pred_train)))}")
    print(f"Test Accuracy: {accuracy_score(list(itertools.chain(*Y_test_list[i])), list(itertools.chain(*pred_test)))}\n")


Function 1:
Solver cost:  2
Training Accuracy: 1.0
Test Accuracy: 0.8

Function 2:
Solver cost:  4
Training Accuracy: 1.0
Test Accuracy: 0.8

Function 3:
Solver cost:  1
Training Accuracy: 1.0
Test Accuracy: 0.8

